# Example: using OpenMM to run a simulation with a NequIP ML potential

You can run this example directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm-ml/blob/mace/examples/nequip/run_mace.ipynb)

## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge() # use mamba on colab because it is faster than conda

## Install software

First install everything we can from [conda-forge](https://conda-forge.org/).
Then use pip.

⚠️ The installation might take up to 10 min!

In [ ]:
#https://github.com/openmm/openmm-torch/issues/88
%env CONDA_OVERRIDE_CUDA=12.0
!mamba install -c conda-forge openmm-torch pytorch=*=cuda* openmmtools

!pip install git+https://github.com/ACEsuit/mace
!pip install git+https://github.com/sef43/openmm-ml@mace

## Get the files we need to run the example

In [ ]:
!wget https://raw.githubusercontent.com/sef43/openmm-ml/mace/examples/mace/mace/openmm-ml/examples/mace/ani500k_small_CC.model

## Run simulation

In [ ]:
import openmm
import openmm.app as app
import openmm.unit as unit
from openmmml import MLPotential
from sys import stdout
from openmmtools.testsystems import AlanineDipeptideVacuum

# Get the system of alanine dipeptide
ala2 = AlanineDipeptideVacuum(constraints=None)

# Remove MM forces
while ala2.system.getNumForces() > 0:
  ala2.system.removeForce(0)

# The system should not contain any additional force and constrains
assert ala2.system.getNumConstraints() == 0
assert ala2.system.getNumForces() == 0

potential = MLPotential('mace', model_path='ani500k_small_CC.model')


system = potential.createSystem(ala2.topology)


integrator = openmm.LangevinIntegrator(300*unit.kelvin, 10.0/unit.picoseconds, 1.0*unit.femtosecond)
simulation=app.Simulation(ala2.topology, system, integrator)
simulation.context.setPositions(ala2.positions)
simulation.reporters.append(app.PDBReporter('output.pdb', 100))
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True))

simulation.step(1000)